## <center>*Long-Minus-Short Portfolios*
1. 对每个因子 制作每个截面上的 *long-minus-short portfolio*
2. 使用 long-leg minus short-leg portfolio return
3. long-leg 和 short-leg portfolio 都使用 *rank-weighted return*

In [1]:
import pandas as pd
import warnings
from tqdm.notebook import tqdm
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv('./cache/BalancedPanel.csv', low_memory=False)

In [3]:
rf = pd.read_csv('./data/F-F_Research_Data_Factors.csv').iloc[:, [0,4]]
rf.columns = ['yyyymm', 'rf']; rf['rf'] /= 100

In [4]:
df['yyyymm'] = df['yyyymm'].astype(int)
rf['yyyymm'] = rf['yyyymm'].astype(int)
df = df.merge(rf, on='yyyymm', how='left')
# turn return into excess return
df['RET'] = df['RET'] - df['rf']
del df['rf']

In [5]:
# 每个因子构建 long-minus-short portfolio 组合截面收益率
# long-leg portfolio 使用 RET 高于中位数样本的 rank-weighted average
# short-leg portfolio 使用 RET 低于中位数样本的 rank-weighted average

factor_cols = [col for col in df.columns if col not in ['yyyymm', 'permno', 'RET']]

def handle_lms(factor_name, yyyymm):
    data = yyyymm[[factor_name, 'RET']].copy()
    long_group = data[data[factor_name] >= data[factor_name].median()]
    short_group = data[data[factor_name] < data[factor_name].median()]
    # long-leg rank-weighted average
    long_weighted_avg = (long_group['RET'] * long_group[factor_name].rank(ascending=True)).sum()\
        / long_group[factor_name].rank(ascending=True).sum()
    # short-leg rank-weighted average
    short_weighted_avg = (short_group['RET'] * short_group[factor_name].rank(ascending=False)).sum()\
        / short_group[factor_name].rank(ascending=False).sum()
    if long_group.empty:
        return -short_weighted_avg
    elif short_group.empty:
        return long_weighted_avg
    return long_weighted_avg - short_weighted_avg


# 写一个函数 输入 yyyymm 返回该截面上的每个因子的 long-short 组合收益率
def handle_cross_section(yyyymm):
    global factor_cols, df
    portfolio_rtns = [yyyymm]
    for factor in factor_cols:
        yyyymm_df = df[df['yyyymm'] == yyyymm]
        portfolio_rtn = handle_lms(factor, yyyymm_df)
        portfolio_rtns.append(portfolio_rtn)
    return portfolio_rtns

In [6]:
long_short_portfolio_rtns = []
for yyyymm in tqdm(df['yyyymm'].unique(), desc='Processing Cross-Sections', colour='blue'):
    long_short_portfolio_rtns.append(handle_cross_section(yyyymm))

long_short_portfolio_rtns = pd.DataFrame(long_short_portfolio_rtns, columns=['yyyymm']+factor_cols)
long_short_portfolio_rtns['yyyymm'] = df['yyyymm'].unique()

Processing Cross-Sections:   0%|          | 0/659 [00:00<?, ?it/s]

In [7]:
long_short_portfolio_rtns

,yyyymm,MaxRet,RealizedVol,IdioVol3F,ReturnSkew3F,High52,ReturnSkew,IdioVolAHT,zerotrade1M,DolVol,...,DivInit,IndIPO,ExchSwitch,DivOmit,Spinoff,FirmAge,IndMom,betaVIX,VolSD,Mom6m
0,197001,-0.022823,-0.021712,-0.014996,-0.056396,0.071570,-0.065887,0.016471,-0.031486,-0.035241,...,0.017074,0.016909,0.024801,0.005470,0.013002,-0.059356,0.007943,-0.004913,-0.015119,0.114313
1,197002,-0.003664,-0.011480,-0.008748,-0.064380,0.134947,-0.072887,0.015311,-0.015795,-0.017999,...,-0.011912,-0.011691,0.037457,0.008600,-0.010427,0.003701,0.017070,-0.004446,-0.015417,0.113456
2,197003,0.029763,-0.002334,-0.005837,-0.074349,0.239440,-0.074835,0.008926,-0.026036,-0.018940,...,-0.060480,-0.059742,0.137510,0.057687,-0.047706,0.117005,0.049380,-0.017822,-0.035985,0.107710
3,197004,-0.007777,-0.025206,-0.028268,-0.077262,0.169261,-0.074618,-0.019653,-0.037155,-0.017177,...,-0.032768,-0.031981,0.054877,-0.031981,-0.025512,0.063894,0.057498,-0.003749,-0.044473,0.117711
4,197005,0.037362,0.022492,0.008892,-0.059927,0.170072,-0.049957,0.009913,-0.021427,-0.020260,...,-0.028564,-0.028162,-0.050876,-0.011446,-0.021607,0.060272,0.063794,-0.009540,-0.020458,0.148859
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
654,202407,0.039772,0.036900,0.023779,0.017277,0.027705,0.018373,0.035582,0.032851,-0.017830,...,-0.003393,-0.002702,-0.026327,-0.085658,-0.002702,-0.001558,0.013956,0.052306,0.011174,0.013186
655,202408,0.032972,0.034728,0.029562,0.014829,0.027237,0.006424,0.041386,0.008148,-0.028561,...,-0.003679,-0.003341,-0.045473,-0.140709,-0.003341,-0.000956,0.000012,0.020603,0.006690,0.003788
656,202409,0.029980,0.030067,0.031553,0.008517,0.011594,0.005588,0.015008,0.007575,-0.007354,...,-0.020625,-0.020395,-0.020047,-0.184424,-0.020395,0.015681,0.034512,0.034884,0.007007,0.025307
657,202410,0.011822,0.015077,0.025644,0.019869,0.051094,0.003758,0.041436,0.021562,-0.019508,...,0.047765,0.047639,-0.017854,-0.118242,0.047639,0.016440,-0.001255,-0.019756,0.033127,0.028541


In [8]:
long_short_portfolio_rtns

,yyyymm,MaxRet,RealizedVol,IdioVol3F,ReturnSkew3F,High52,ReturnSkew,IdioVolAHT,zerotrade1M,DolVol,...,DivInit,IndIPO,ExchSwitch,DivOmit,Spinoff,FirmAge,IndMom,betaVIX,VolSD,Mom6m
0,197001,-0.022823,-0.021712,-0.014996,-0.056396,0.071570,-0.065887,0.016471,-0.031486,-0.035241,...,0.017074,0.016909,0.024801,0.005470,0.013002,-0.059356,0.007943,-0.004913,-0.015119,0.114313
1,197002,-0.003664,-0.011480,-0.008748,-0.064380,0.134947,-0.072887,0.015311,-0.015795,-0.017999,...,-0.011912,-0.011691,0.037457,0.008600,-0.010427,0.003701,0.017070,-0.004446,-0.015417,0.113456
2,197003,0.029763,-0.002334,-0.005837,-0.074349,0.239440,-0.074835,0.008926,-0.026036,-0.018940,...,-0.060480,-0.059742,0.137510,0.057687,-0.047706,0.117005,0.049380,-0.017822,-0.035985,0.107710
3,197004,-0.007777,-0.025206,-0.028268,-0.077262,0.169261,-0.074618,-0.019653,-0.037155,-0.017177,...,-0.032768,-0.031981,0.054877,-0.031981,-0.025512,0.063894,0.057498,-0.003749,-0.044473,0.117711
4,197005,0.037362,0.022492,0.008892,-0.059927,0.170072,-0.049957,0.009913,-0.021427,-0.020260,...,-0.028564,-0.028162,-0.050876,-0.011446,-0.021607,0.060272,0.063794,-0.009540,-0.020458,0.148859
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
654,202407,0.039772,0.036900,0.023779,0.017277,0.027705,0.018373,0.035582,0.032851,-0.017830,...,-0.003393,-0.002702,-0.026327,-0.085658,-0.002702,-0.001558,0.013956,0.052306,0.011174,0.013186
655,202408,0.032972,0.034728,0.029562,0.014829,0.027237,0.006424,0.041386,0.008148,-0.028561,...,-0.003679,-0.003341,-0.045473,-0.140709,-0.003341,-0.000956,0.000012,0.020603,0.006690,0.003788
656,202409,0.029980,0.030067,0.031553,0.008517,0.011594,0.005588,0.015008,0.007575,-0.007354,...,-0.020625,-0.020395,-0.020047,-0.184424,-0.020395,0.015681,0.034512,0.034884,0.007007,0.025307
657,202410,0.011822,0.015077,0.025644,0.019869,0.051094,0.003758,0.041436,0.021562,-0.019508,...,0.047765,0.047639,-0.017854,-0.118242,0.047639,0.016440,-0.001255,-0.019756,0.033127,0.028541


In [9]:
long_short_portfolio_rtns.describe()

,yyyymm,MaxRet,RealizedVol,IdioVol3F,ReturnSkew3F,High52,ReturnSkew,IdioVolAHT,zerotrade1M,DolVol,...,DivInit,IndIPO,ExchSwitch,DivOmit,Spinoff,FirmAge,IndMom,betaVIX,VolSD,Mom6m
count,659.000000,659.000000,659.000000,659.000000,659.000000,659.000000,659.000000,659.000000,659.000000,659.000000,...,659.000000,659.000000,659.000000,659.000000,659.000000,659.000000,659.000000,659.000000,659.000000,659.000000
mean,199702.394537,0.005193,0.000031,0.000366,-0.005509,0.025126,-0.006470,0.001866,-0.003807,-0.005117,...,0.008523,0.006626,0.003180,-0.045397,0.008652,0.006787,0.004483,0.003017,0.001135,0.022920
std,1586.357601,0.036328,0.041692,0.037994,0.019622,0.054731,0.022927,0.040148,0.026793,0.028120,...,0.043383,0.050645,0.059142,0.112368,0.042404,0.026897,0.030580,0.015882,0.023249,0.044387
min,197001.000000,-0.207210,-0.229628,-0.204866,-0.087843,-0.307879,-0.091904,-0.211212,-0.090180,-0.111052,...,-0.261883,-0.198524,-0.333615,-0.836489,-0.262427,-0.136653,-0.173302,-0.083794,-0.099342,-0.308478
25%,198309.500000,-0.012682,-0.022712,-0.020189,-0.013817,-0.003979,-0.017592,-0.018217,-0.022264,-0.021843,...,-0.015117,-0.020909,-0.025569,-0.116967,-0.013642,-0.007417,-0.010423,-0.005918,-0.012881,-0.000240
50%,199706.000000,0.005442,0.001265,0.002778,-0.002166,0.020698,-0.001959,0.003996,-0.003055,-0.006572,...,0.010073,0.008879,0.006352,-0.013374,0.010536,0.005259,0.006286,0.001666,0.000995,0.019067
75%,201102.500000,0.027160,0.026803,0.024391,0.006941,0.051985,0.007577,0.027651,0.015032,0.009712,...,0.034737,0.034330,0.031023,0.026010,0.034294,0.019203,0.020701,0.010061,0.016048,0.047607
max,202411.000000,0.116580,0.160518,0.119080,0.036724,0.239440,0.045987,0.132637,0.083864,0.139704,...,0.187464,0.217842,0.287663,0.244852,0.187812,0.137504,0.123841,0.087797,0.071949,0.148859


In [10]:
long_short_portfolio_rtns.to_csv('./cache/LongShortLegRtns.csv', index=False)